In [1]:
from utils import config, data
from utils.batcher import *
from utils.train_util import *
from utils.initialize import loadCheckpoint, save_model

from utils.write_result import *
from datetime import datetime as dt
from tqdm import tqdm
# from beam.transormer_beam_search import *
# from beam.transormer_beam_search_hyp_order import *
# from beam.transormer_beam_search_beam_order import *
from beam.transormer_beam_search_multi_order import *
from tensorboardX import SummaryWriter
import argparse
from utils.rl_util import *
from torch.distributions import Categorical

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

os.environ['CUDA_LAUNCH_BLOCKING'] = "1" 

parser = argparse.ArgumentParser()
parser.add_argument('--key_attention', type=bool, default=False, help = 'True/False')# for PG
parser.add_argument('--intra_encoder', type=bool, default=False, help = 'True/False')# for PG
parser.add_argument('--intra_decoder', type=bool, default=False, help = 'True/False')# for PG
parser.add_argument('--copy', type=bool, default=False, help = 'True/False') # for transformer
parser.add_argument('--transformer', type=bool, default=True, help = 'True/False')
parser.add_argument('--train_rl', type=bool, default=False, help = 'True/False')
parser.add_argument('--keywords', type=str, default='POS_FOP_keywords', 
                    help = 'POS_FOP_keywords / DEP_FOP_keywords / TextRank_keywords')

parser.add_argument('--lr', type=float, default=0.0001)
parser.add_argument('--rand_unif_init_mag', type=float, default=0.02)
parser.add_argument('--trunc_norm_init_std', type=float, default=0.001)
parser.add_argument('--mle_weight', type=float, default=1.0)
parser.add_argument('--gound_truth_prob', type=float, default=0.1)

parser.add_argument('--max_enc_steps', type=int, default=1000)
parser.add_argument('--max_dec_steps', type=int, default=50)
parser.add_argument('--min_dec_steps', type=int, default=8)
parser.add_argument('--max_epochs', type=int, default=20)
parser.add_argument('--vocab_size', type=int, default=50000)
parser.add_argument('--beam_size', type=int, default=16)
parser.add_argument('--batch_size', type=int, default=8)

parser.add_argument('--hidden_dim', type=int, default=512)
parser.add_argument('--emb_dim', type=int, default=512)
parser.add_argument('--gradient_accum', type=int, default=1)

parser.add_argument('--load_ckpt', type=str, default=None, help='0002000')
parser.add_argument('--word_emb_type', type=str, default='word2Vec', help='word2Vec/glove/FastText')
parser.add_argument('--pre_train_emb', type=bool, default=False, help = 'True/False') # 若pre_train_emb為false, 則emb type為NoPretrain

opt = parser.parse_args(args=[])
config = re_config(opt)
# config.copy = True
loggerName, writerPath = getName(config)    
logger = getLogger(loggerName)
writer = SummaryWriter(writerPath)

I0417 23:19:08.661403 139696977332032 file_utils.py:35] PyTorch version 1.4.0 available.
2020-04-17 23:19:09 - Transformer_no_pretrain - INFO: - logger已啟動
I0417 23:19:09.506498 139696977332032 train_util.py:146] logger已啟動


In [2]:
train_loader, validate_loader, vocab = getDataLoader(logger, config)

2020-04-17 23:19:15 - Transformer_no_pretrain - INFO: - train : 37771, test : 4197
I0417 23:19:15.827946 139696977332032 batcher.py:171] train : 37771, test : 4197
2020-04-17 23:19:15 - Transformer_no_pretrain - INFO: - train batches : 4722, test batches : 525
I0417 23:19:15.846394 139696977332032 batcher.py:182] train batches : 4722, test batches : 525


In [3]:
from transformer import Model
import torch.nn as nn
import torch as T
import torch.nn.functional as F
from torch.nn.utils import clip_grad_norm_

load_step = None
model = Model(pre_train_emb=config.pre_train_emb, 
              word_emb_type = config.word_emb_type, 
              vocab = vocab,config=config)

model = model.cuda()

optimizer = T.optim.Adam(model.parameters(), lr=config.lr)   
# optimizer = T.optim.Adagrad(model.parameters(),lr=config.lr, initial_accumulator_value=0.1)

load_model_path = config.save_model_path + '/%s/%s.tar' % (logger, config.load_ckpt)

if os.path.exists(load_model_path):
    model, optimizer, load_step = loadCheckpoint(logger, load_model_path, model, optimizer)

In [4]:
def train_one(model, config, batch):
        ''' Calculate Negative Log Likelihood Loss for the given batch. In order to reduce exposure bias,
                pass the previous generated token as input with a probability of 0.25 instead of ground truth label
        Args:
        :param enc_out: Outputs of the encoder for all time steps (batch_size, length_input_sequence, 2*hidden_size)
        :param enc_hidden: Tuple containing final hidden state & cell state of encoder. Shape of h & c: (batch_size, hidden_size)
        :param enc_padding_mask: Mask for encoder input; Tensor of size (batch_size, length_input_sequence) with values of 0 for pad tokens & 1 for others
        :param ct_e: encoder context vector for time_step=0 (eq 5 in https://arxiv.org/pdf/1705.04304.pdf)
        :param extra_zeros: Tensor used to extend vocab distribution for pointer mechanism
        :param enc_batch_extend_vocab: Input batch that stores OOV ids
        :param batch: batch object
        '''
        'Encoder data'
        enc_batch, enc_padding_mask, enc_lens, enc_batch_extend_vocab, extra_zeros, _, \
        _, _, _, _= \
            get_input_from_batch(batch, config, batch_first = True)
       
        'Decoder data'
        dec_batch, dec_padding_mask, dec_lens, max_dec_len, target_batch = \
        get_output_from_batch(batch, config, batch_first = True) # Get input and target batchs for training decoder

#         print(enc_batch_extend_vocab)
        pred = model(enc_batch, dec_batch, enc_padding_mask, dec_padding_mask, enc_batch_extend_vocab, extra_zeros)
        loss = model.label_smoothing_loss(pred, target_batch)
#         loss = model.nll_loss(pred, target_batch, dec_lens)
#         print('loss',loss)#         
        # >>>>>>>> DEBUG Session <<<<<<<<<
#         print('------------------------------------')
#         print("ENC\n")
#         print(enc_batch.shape)
#         print("DEC\n")
#         print(dec_batch.shape)
        # print("TGT\n")
        # print(target_batch.shape)
        # print("ENCP\n")
        # print(enc_padding_mask.shape)
        # print("DECP\n")
        # print(dec_padding_mask.shape)
        return loss


In [5]:
# @torch.no_grad()
@torch.autograd.no_grad()
def validate(validate_loader, config, model):
    model.eval()
    losses = []
#     batch = next(iter(validate_loader))
    for batch in validate_loader:
        loss = train_one(model, config, batch)
        losses.append(loss.item())
#         break
    model.train()
    ave_loss = sum(losses) / len(losses)
    return ave_loss

In [6]:
@torch.autograd.no_grad()
def calc_running_avg_loss(loss, running_avg_loss, decay=0.99):
    if running_avg_loss == 0:  # on the first iteration just take the loss
        running_avg_loss = loss
    else:
        running_avg_loss = running_avg_loss * decay + (1 - decay) * loss
    running_avg_loss = min(running_avg_loss, 12)  # clip
    return running_avg_loss

In [7]:
from random import randint
@torch.autograd.no_grad()
def decode(writer, logger, step, config, model, batch, mode):
    # 動態取batch
    model.eval()
    model.is_predicting = True
    if mode == 'test':
        num = len(iter(batch))
        select_batch = None
        rand_b_id = randint(0,num-2)
#         logger.info('test_batch : ' + str(num)+ ' ' + str(rand_b_id))
        for idx, b in enumerate(batch):
            if idx == rand_b_id:
                select_batch = b
                break   
#         select_batch = iter(batch)[rand_b_id]
#         select_batch = next(iter(batch))
        batch = select_batch
        if type(batch) == torch.utils.data.dataloader.DataLoader:
            batch = next(iter(batch))

#     print(batch.enc_pad_mask.shape[0])
    pred_ids = beam_search(config, batch, model, START, END, UNKNOWN_TOKEN)
    model.is_predicting = False

    article_sents, decoded_sents, keywords_list, \
    ref_sents, long_seq_index = prepare_result(vocab, batch, pred_ids)
#     print(prepare_result(vocab, batch, pred_ids))
    rouge_1, rouge_2, rouge_l = write_rouge(writer, step, mode,article_sents, decoded_sents, \
                keywords_list, ref_sents, long_seq_index)

    write_bleu(writer, step, mode, article_sents, decoded_sents, \
               keywords_list, ref_sents, long_seq_index)

    write_group(writer, step, mode, article_sents, decoded_sents,\
                keywords_list, ref_sents, long_seq_index)

    return rouge_l

In [8]:
from random import randint
@torch.autograd.no_grad()
def avg_acc(writer, logger, epoch, config, model, dataloader, mode):
    # 動態取batch
    num = len(iter(dataloader))
    avg_rouge_l = []
    model.eval()
    model.is_predicting = True
    for idx, batch in enumerate(dataloader): 
        if batch.enc_inp.shape[0] != config.batch_size: continue
        pred_ids = beam_search(config, batch, model, START, END, UNKNOWN_TOKEN)
        
        article_sents, decoded_sents, keywords_list, \
        ref_sents, long_seq_index = prepare_result(vocab, batch, pred_ids)

        rouge_1, rouge_2, rouge_l = write_rouge(writer, None, None, article_sents, decoded_sents, \
                    keywords_list, ref_sents, long_seq_index, write = False)
        avg_rouge_l.append(rouge_l)

    model.is_predicting = False
    avg_rouge_l = sum(avg_rouge_l) / num
    writer.add_scalars('scalar_avg/acc',  
                   {'%sing_avg_acc'%(mode): avg_rouge_l
                   }, epoch)

    return avg_rouge_l

In [9]:
def RL(model, config, batch, greedy):    
        '''Generate sentences from decoder entirely using sampled tokens as input. These sentences are used for ROUGE evaluation
        Args
        :param enc_out: Outputs of the encoder for all time steps (batch_size, length_input_sequence, 2*hidden_size)
        :param enc_hidden: Tuple containing final hidden state & cell state of encoder. Shape of h & c: (batch_size, hidden_size)
        :param enc_padding_mask: Mask for encoder input; Tensor of size (batch_size, length_input_sequence) with values of 0 for pad tokens & 1 for others
        :param ct_e: encoder context vector for time_step=0 (eq 5 in https://arxiv.org/pdf/1705.04304.pdf)
        :param extra_zeros: Tensor used to extend vocab distribution for pointer mechanism
        :param enc_batch_extend_vocab: Input batch that stores OOV ids
        :param article_oovs: Batch containing list of OOVs in each example
        :param greedy: If true, performs greedy based sampling, else performs multinomial sampling
        Returns:
        :decoded_strs: List of decoded sentences
        :log_probs: Log probabilities of sampled words
        '''
        'Encoder data'
        enc_batch, enc_padding_mask, enc_lens, enc_batch_extend_vocab, extra_zeros, _, \
        _, _, _, _= \
            get_input_from_batch(batch, config, batch_first = True)

        'Decoder data'
        dec_batch, dec_padding_mask, dec_lens, max_dec_len, target_batch = \
        get_output_from_batch(batch, config, batch_first = True) # Get input and target batchs for training decoder
        log_probs = []                          #Stores log probabilites of generated samples

        probs = model(enc_batch, dec_batch, enc_padding_mask, dec_padding_mask, enc_batch_extend_vocab, extra_zeros)
        dec_padding_mask = dec_padding_mask.eq(0).float()
        if greedy is False:
                multi_dist = Categorical(probs) # 建立以參數probs為標準的類別分佈
                # perform multinomial sampling
                inds = multi_dist.sample()  # 將下一個時間點的x_t，視為下一個action   
                # 使用log_prob实施梯度方法 Policy Gradient，构造一个等价類別分佈的损失函数
                log_probs = multi_dist.log_prob(inds)  
        else:
            # perform greedy sampling distribution
            _, inds = T.max(probs, dim=1)  # 因greedy以機率最大進行取樣，視為其中一個action 

        inds = inds.detach() # detach返回的 Variable 永远不会需要梯度

        if greedy is False:                                                                         #If multinomial based sampling, compute log probabilites of sampled words
            log_probs = log_probs * dec_padding_mask # 遮罩掉為[END] or [STOP]不計算損失           #Not considering sampled words with padding mask = 0
            lens = T.sum(dec_padding_mask, dim=1) # 計算每個sample words生成的總長度               #Length of sampled sentence
            log_probs = T.sum(log_probs, dim=1) / lens  # 計算平均的每個句子的log loss # (bs,1)        #compute normalizied log probability of a sentence

        decoded_strs = []
        for i in range(len(enc_batch)):
            id_list = inds[i].cpu().numpy() # 取出每個sample sentence 的word id list
            S = output2words(id_list, vocab, batch.art_oovs[i]) #Generate sentence corresponding to sampled words
            try:
                end_idx = S.index(data.STOP_DECODING)
                S = S[:end_idx]
            except ValueError:
                S = S
            if len(S) < 2:          #If length of sentence is less than 2 words, replace it with "xxx"; Avoids setences like "." which throws error while calculating ROUGE
                S = ["xxx"]
            S = " ".join(S)
            decoded_strs.append(S)
        return decoded_strs, log_probs

In [10]:
def train_one_RL(model, config, batch):
    # Self-Critical sequence training(SCST)
    sample_sents, RL_log_probs = RL(model, config, batch, greedy=False)   # multinomial sampling
    with T.autograd.no_grad():        
        greedy_sents, _ = RL(model, config, batch, greedy=True)  # greedy sampling

    sample_reward = reward_function(sample_sents, batch.original_abstract) # r(w^s):通过根据概率来随机sample词生成句子的reward值
    baseline_reward = reward_function(greedy_sents, batch.original_abstract) # r(w^):测试阶段使用greedy decoding取概率最大的词来生成句子的reward值

    batch_reward = T.mean(sample_reward).item()
    #Self-critic policy gradient training (eq 15 in https://arxiv.org/pdf/1705.04304.pdf)
    rl_loss = -(sample_reward - baseline_reward) * RL_log_probs  # SCST梯度計算公式     
    rl_loss = T.mean(rl_loss)  
    '''
    公式的意思就是：对于如果当前sample到的词比测试阶段生成的词好，那么在这次词的维度上，整个式子的值就是负的（因为后面那一项一定为负），
    这样梯度就会上升，从而提高这个词的分数st；而对于其他词，后面那一项为正，梯度就会下降，从而降低其他词的分数
    '''                 
    return rl_loss, batch_reward

In [11]:
import pandas as pd
import time
from utils.write_result import *

@torch.autograd.no_grad()
def decode_write_all(writer, logger, epoch, config, model, dataloader, mode):
    # 動態取batch
    num = len(iter(dataloader))
    avg_rouge_1, avg_rouge_2, avg_rouge_l,  = [], [], []
    avg_bleu1, avg_bleu2, avg_bleu3, avg_bleu4 = [], [], [], []
    outFrame = None
    avg_time = 0
    for idx, batch in enumerate(dataloader):
        start = time.time() 
#         'Encoder data'
#         enc_batch, enc_padding_mask, enc_lens, enc_batch_extend_vocab, extra_zeros, coverage, \
#         ct_e, enc_key_batch, enc_key_mask, enc_key_lens= \
#             get_input_from_batch(batch, config, batch_first = True)

#         enc_batch = model.embeds(enc_batch)  # Get embeddings for encoder input    
#         enc_key_batch = model.embeds(enc_key_batch)  # Get key embeddings for encoder input

#         enc_out, enc_hidden = model.encoder(enc_batch, enc_lens)

        if batch.enc_inp.shape[0] != config.batch_size: continue
        pred_ids = beam_search(config, batch, model, START, END, UNKNOWN_TOKEN)
        
#         'Feed encoder data to predict'
#         pred_ids = beam_search(enc_hidden, enc_out, enc_padding_mask, ct_e, extra_zeros, 
#                                 enc_batch_extend_vocab, enc_key_batch, enc_key_lens, model, 
#                                 START, END, UNKNOWN_TOKEN)

        article_sents, decoded_sents, keywords_list, ref_sents, long_seq_index = prepare_result(vocab, batch, pred_ids)
        cost = (time.time() - start)
        avg_time += cost
        logger.info('decode batch cost time : %s ms'%(cost / (config.batch_size)))        
        # ----------------------------------------------------
        batch_frame = {
            'article':article_sents,
            'keywords':keywords_list,
            'reference':ref_sents,
            'decoded':decoded_sents
        }
        batch_frame = pd.DataFrame(batch_frame)
        if idx == 0: outFrame = batch_frame 
        else: outFrame = pd.concat([outFrame, batch_frame], axis=0, ignore_index=True) 
        # ----------------------------------------------------
        rouge_1, rouge_2, rouge_l = write_rouge(writer, None, None, article_sents, decoded_sents, \
                    keywords_list, ref_sents, long_seq_index, write = False)
        Bleu_1, Bleu_2, Bleu_3, Bleu_4 = write_bleu(writer, None, None, article_sents, decoded_sents, \
            keywords_list, ref_sents, long_seq_index, write = False)
        # ----------------------------------------------------
        avg_rouge_1.append(rouge_1)
        avg_rouge_2.append(rouge_2)
        avg_rouge_l.append(rouge_l)
        
        avg_bleu1.append(Bleu_1)
        avg_bleu2.append(Bleu_2)
        avg_bleu3.append(Bleu_3)
        avg_bleu4.append(Bleu_4)
        # ----------------------------------------------------
    avg_rouge_1 = sum(avg_rouge_1) / num
    avg_rouge_2 = sum(avg_rouge_2) / num
    avg_rouge_l = sum(avg_rouge_l) / num
    writer.add_scalars('Rouge_avg/mode',  
                    {'avg_rouge_1': avg_rouge_1,
                    'avg_rouge_2': avg_rouge_2,
                    'avg_rouge_l': avg_rouge_l
                    }, epoch)
    # --------------------------------------               
    avg_bleu1 = sum(avg_bleu1)/num
    avg_bleu2 = sum(avg_bleu2)/num
    avg_bleu3 = sum(avg_bleu3)/num
    avg_bleu4 = sum(avg_bleu4)/num
    
    writer.add_scalars('BLEU_avg/mode',  
                    {
                    '%sing_avg_bleu1'%(mode): avg_bleu1,
                    '%sing_avg_bleu1'%(mode): avg_bleu2,
                    '%sing_avg_bleu1'%(mode): avg_bleu3,
                    '%sing_avg_bleu1'%(mode): avg_bleu4,                   
                    }, epoch)
    # --------------------------------------      
    outFrame.to_excel(writerPath + '/%s_output.xls'% mode)
    avg_time = avg_time / (num * config.batch_size) 
    with open(writerPath + '/%s_res.txt'% mode, 'w', encoding='utf-8') as f:
        f.write('Accuracy result:\n')
        f.write('##-- Rouge --##\n')
        f.write('%sing_avg_rouge_1: %s \n'%(mode, avg_rouge_1))
        f.write('%sing_avg_rouge_2: %s \n'%(mode, avg_rouge_2))
        f.write('%sing_avg_rouge_l: %s \n'%(mode, avg_rouge_l))

        f.write('##-- BLEU --##\n')
        f.write('%sing_avg_bleu1: %s \n'%(mode, avg_bleu1))
        f.write('%sing_avg_bleu2: %s \n'%(mode, avg_bleu2))
        f.write('%sing_avg_bleu3: %s \n'%(mode, avg_bleu3))
        f.write('%sing_avg_bleu4: %s \n'%(mode, avg_bleu4))

        f.write('Execute Time: %s \n' % avg_time)        
    # --------------------------------------              
    return avg_rouge_l

In [12]:
write_train_para(writer, config)
logger.info('------Training START--------')
running_avg_loss, running_avg_rl_loss = 0, 0
sum_total_reward = 0
step = 0

try:
    for epoch in range(config.max_epochs):
        for batch in train_loader:
            step += 1
            mle_loss = train_one(model, config, batch)
            if config.train_rl:
                rl_loss, batch_reward = train_one_RL(model, config, batch)             
                writer.add_scalars('scalar/RL_Loss',  
                       {'rl_loss': rl_loss
                       }, step)
                writer.add_scalars('scalar/Reward',  
                       {'batch_reward': batch_reward
                       }, step)
                
                if step%1000 == 0 :
                    logger.info('epoch %d: %d, RL_Loss = %f, batch_reward = %f'
                                    % (epoch, step, rl_loss, batch_reward))
                sum_total_reward += batch_reward
            else:
                rl_loss = T.FloatTensor([0]).cuda()
            (config.mle_weight * mle_loss + config.rl_weight * rl_loss).backward()  # 反向传播，计算当前梯度

            '''梯度累加就是，每次获取1个batch的数据，计算1次梯度，梯度不清空'''
            if step % (config.gradient_accum) == 0: # gradient accumulation
    #             clip_grad_norm_(model.parameters(), 5.0)                      
                optimizer.step() # 根据累计的梯度更新网络参数
                optimizer.zero_grad() # 清空过往梯度 

            if step%1000 == 0 :
                with T.autograd.no_grad():
                    train_batch_loss = mle_loss.item()
                    train_batch_rl_loss = rl_loss.item()
                    val_avg_loss = validate(validate_loader, config, model) # call batch by validate_loader
                    running_avg_loss = calc_running_avg_loss(train_batch_loss, running_avg_loss)
                    running_avg_rl_loss = calc_running_avg_loss(train_batch_rl_loss, running_avg_rl_loss)
                    running_avg_reward = sum_total_reward / step
                    logger.info('epoch %d: %d, training batch loss = %f, running_avg_loss loss = %f, validation loss = %f'
                                % (epoch, step, train_batch_loss, running_avg_loss, val_avg_loss))
                    writer.add_scalars('scalar/Loss',  
                       {'train_batch_loss': train_batch_loss
                       }, step)
                    writer.add_scalars('scalar_avg/loss',  
                       {'train_avg_loss': running_avg_loss,
                        'test_avg_loss': val_avg_loss
                       }, step)
                    if running_avg_reward > 0:
                        logger.info('epoch %d: %d, running_avg_reward = %f'
                                % (epoch, step, running_avg_reward))
                        writer.add_scalars('scalar_avg/Reward',  
                           {'running_avg_reward': running_avg_reward
                           }, step)
                    if running_avg_rl_loss != 0:
                        logger.info('epoch %d: %d, running_avg_rl_loss = %f'
                                % (epoch, step, running_avg_rl_loss))
                        writer.add_scalars('scalar_avg/RL_Loss',  
                           {'running_avg_rl_loss': running_avg_rl_loss
                           }, step)

            if step%5000 == 0:
                save_model(config, logger, model, optimizer, step, vocab, running_avg_loss, \
                           r_loss=0, title = loggerName)
            if step%1000 == 0 and step > 0:
                train_rouge_l_f = decode(writer, logger, step, config, model, batch, mode = 'train') # call batch by validate_loader
                test_rouge_l_f = decode(writer, logger, step, config, model, validate_loader, mode = 'test') # call batch by validate_loader

                writer.add_scalars('scalar/Rouge-L',  
                   {'train_rouge_l_f': train_rouge_l_f,
                    'test_rouge_l_f': test_rouge_l_f
                   }, step)
                logger.info('epoch %d: %d, train_rouge_l_f = %f, test_rouge_l_f = %f'
                                % (epoch, step, train_rouge_l_f, test_rouge_l_f))
#         break
#         train_avg_acc = avg_acc(writer, logger, epoch, config, model, train_loader, mode = 'train')
#         test_avg_acc = avg_acc(writer, logger, epoch, config, model, validate_loader, mode = 'test')
#         logger.info('epoch %d: %d, train_avg_acc = %f, test_avg_acc = %f' % (epoch, step, train_avg_acc, test_avg_acc))
except Excepation as e:
        print(e)
else:
    logger.info(u'------Training SUCCESS--------')  
finally:
    logger.info(u'------Training END--------')    
    train_avg_acc = decode_write_all(writer, logger, epoch, config, model, validate_loader, mode = 'train')
    test_avg_acc = decode_write_all(writer, logger, epoch, config, model, validate_loader, mode = 'test')
    logger.info('epoch %d: train_avg_acc = %f, test_avg_acc = %f' % (epoch, train_avg_acc, test_avg_acc))
    removeLogger(logger)


2020-04-17 23:19:20 - Transformer_no_pretrain - INFO: - ------Training START--------
I0417 23:19:20.415174 139696977332032 <ipython-input-12-855e57fc9e57>:2] ------Training START--------
2020-04-17 23:24:39 - Transformer_no_pretrain - INFO: - epoch 0: 1000, training batch loss = 4.455187, running_avg_loss loss = 4.455187, validation loss = 4.558357
I0417 23:24:39.946844 139696977332032 <ipython-input-12-855e57fc9e57>:44] epoch 0: 1000, training batch loss = 4.455187, running_avg_loss loss = 4.455187, validation loss = 4.558357


predicted_words [[181, 231], [108, 61], [122, 393], [149, 618], [24, 35], [7, 149], [181, 181], [393, 108]]
---------------------------


2020-04-17 23:24:40 - Transformer_no_pretrain - INFO: - epoch 0: 1000, train_rouge_l_f = 0.007919, test_rouge_l_f = 0.034722
I0417 23:24:40.523661 139696977332032 <ipython-input-12-855e57fc9e57>:77] epoch 0: 1000, train_rouge_l_f = 0.007919, test_rouge_l_f = 0.034722


predicted_words [[9, 231], [181, 231], [122, 393], [149, 618], [24, 35], [7, 108], [24, 35], [24, 35]]
---------------------------


2020-04-17 23:29:43 - Transformer_no_pretrain - INFO: - epoch 0: 2000, training batch loss = 4.483906, running_avg_loss loss = 4.455474, validation loss = 4.365143
I0417 23:29:43.283313 139696977332032 <ipython-input-12-855e57fc9e57>:44] epoch 0: 2000, training batch loss = 4.483906, running_avg_loss loss = 4.455474, validation loss = 4.365143


...

2020-04-17 23:46:45 - Transformer_no_pretrain - INFO: - decode batch cost time : 0.03533834218978882 ms
I0417 23:46:45.219948 139696977332032 <ipython-input-11-3340543714f4>:36] decode batch cost time : 0.03533834218978882 ms


predicted_words [[24, 393], [4, 393], [24, 393], [449, 181], [461, 4], [108, 449], [15, 4], [7, 181]]
---------------------------


2020-04-17 23:46:45 - Transformer_no_pretrain - INFO: - decode batch cost time : 0.03495985269546509 ms
I0417 23:46:45.513397 139696977332032 <ipython-input-11-3340543714f4>:36] decode batch cost time : 0.03495985269546509 ms


predicted_words [[24, 393], [4, 393], [24, 393], [449, 181], [122, 461], [231, 9], [122, 461], [7, 181]]
---------------------------


2020-04-17 23:46:45 - Transformer_no_pretrain - INFO: - epoch 0: train_avg_acc = 0.021689, test_avg_acc = 0.021689
I0417 23:46:45.867162 139696977332032 <ipython-input-12-855e57fc9e57>:90] epoch 0: train_avg_acc = 0.021689, test_avg_acc = 0.021689
2020-04-17 23:46:45 - Transformer_no_pretrain - INFO: - logger已關閉
I0417 23:46:45.868928 139696977332032 train_util.py:150] logger已關閉
